<a href="https://colab.research.google.com/github/MichaelDeng03/GenAI_Agents/blob/main/all_agents_tutorials/art_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive Art Agent
This tutorial will walk through the creation of an AI Agent that helps you explore and learn art!

### Overview
For knowledge retrieval, this tutorial uses [LightRAG](https://github.com/HKUDS/LightRAG) which is knowledge graph based RAG implementation. Compared to GraphRAG, LightRAG is more accurate for single topic datasets, and uses 1,000x less tokens for retrieval. Furthermore, documents can be added to LightRAG incrementally without rebuilding the knowledge graph. See the full results in the [paper](https://arxiv.org/abs/2410.05779).

For the agent itself, we use LangGraph.

### Outline
0. [Setup](#setup)
1. [Data Preparation + RAG](#data-preparation--rag)
2. [Agent Creation](#agent-creation)
3. [Final App](#final-app)

### Setup

In [7]:
!uv pip install lightrag-hku aioboto3 numpy ollama openai tenacity aioboto3 transformers torch tiktoken nano_vectordb neo4j oracledb textract setuptools pyvis sqlalchemy langchain_community langchain_openai sqlmodel ipywidgets==7.7.1 asyncio

Using Python 3.12.6 environment at /Users/kentgang/git2/GenAI_Agents/.venv
Resolved 180 packages in 275ms                                       
⠙ Preparing packages... (0/5)                                                   
⠙ Preparing packages... (0/5)----------     0 B/25.73 KiB               
ipython-genutils ------------------------------     0 B/25.73 KiB
⠙ Preparing packages... (0/5)------------     0 B/1.48 MiB              
ipython-genutils ------------------------------     0 B/25.73 KiB
ipywidgets ------------------------------     0 B/120.49 KiB
⠙ Preparing packages... (0/5)------------     0 B/1.48 MiB              
ipython-genutils ------------------------------     0 B/25.73 KiB
ipywidgets ------------------------------ 16.00 KiB/120.49 KiB
⠙ Preparing packages... (0/5)------------     0 B/1.48 MiB              
ipython-genutils ------------------------------ 16.00 KiB/25.73 KiB
ipywidgets ------------------------------ 16.00 KiB/120.49 KiB
⠙ Preparing packages... (0/5)

In [8]:
import asyncio
from ipywidgets import IntSlider, Output
slider = IntSlider()
out = Output()

async def f():
    for i in range(10):
        out.append_stdout('did work ' + str(i) + '\n')
        x = await wait_for_change(slider, 'value')
        out.append_stdout('async function continued with value ' + str(x) + '\n')
asyncio.ensure_future(f())

slider

IntSlider(value=0)

In [5]:
w = input()

In [6]:
w

'faklsdjlfk'

In [9]:
import ipywidgets as widgets
from IPython.display import display
w = widgets.IntSlider()
display(w)

IntSlider(value=0)

In [3]:
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, TwoByTwoLayout

def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

top_left_button = create_expanded_button("Top left", 'info')
top_right_button = create_expanded_button("Top right", 'success')
top_left_text = IntText(description='Top left', layout=Layout(width='auto', height='auto'))
top_right_text = IntText(description='Top right', layout=Layout(width='auto', height='auto'))

TwoByTwoLayout(top_left=top_left_button,
               top_right=top_right_button)

TwoByTwoLayout(children=(Button(button_style='info', description='Top left', layout=Layout(grid_area='top-left…

### Data Preparation + RAG

### Agent Creation

### Final App